In [29]:
import csv
import time
import os
import json
import ast
import pickle
import matplotlib.pylab as plt
import pandas as pd
from itertools import combinations
import math
import pandas as pd
from tqdm import tqdm
import datetime
from dateutil.relativedelta import relativedelta
from scipy.stats.stats import pearsonr

/var/folders/bw/w2pkl8js06ngvf9n2yn4j_tr0000gn/T/ipykernel_47931/3790717874.py:15: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [30]:
author_df = pd.read_csv('/Users/shemingrong/PROJECT1_gender_differences/aps/authors.csv')

author_names_df = pd.read_csv('/Users/shemingrong/PROJECT1_gender_differences/aps/author_names.csv')

genders_df = pd.read_csv('/Users/shemingrong/PROJECT1_gender_differences/aps/genders.csv')



In [31]:
author_names_dict = {} 
author_id_dict = {} 
with open('/Users/shemingrong/PROJECT1_gender_differences/aps/author_names.csv', 'r') as f:
    for i, row in enumerate(csv.reader(f)):
        if i != 0:
            author_names_dict[row[0]] = row[2]
            author_id_dict[row[0]] = row[1]
print(len(author_names_dict))

# extract {id_author: gender}
gender_id = {'0': 'unknown', '1': 'female', '2': 'male'}
author_gender_dict = {}
with open('/Users/shemingrong/Desktop/Jan_dateset/aps/authors.csv', 'r') as f:
    for i, row in enumerate(csv.reader(f)):
        if i != 0:
            if row[0] not in author_gender_dict:
                author_gender_dict[row[0]] = gender_id[row[1]]                
print(len(author_gender_dict))

949944
875729


In [32]:
publications_df = pd.read_csv('/Users/shemingrong/PROJECT1_gender_differences/aps/publications.csv')
publication_id_dict = {}
publication_year_dict = {}
publication_id = {}
with open('/Users/shemingrong/PROJECT1_gender_differences/aps/publications.csv', 'r') as f:
    for i, row in enumerate(csv.reader(f)):
        if i != 0:
            publication_id_dict[row[0]] = row[3]
            publication_year_dict[row[0]] = row[2] 
            publication_id[row[3]] = row[0]

In [33]:
code_id_journal = {}
with open('/Users/shemingrong/PROJECT1_gender_differences/aps/journals.csv', 'r') as f:
    for i, row in enumerate(csv.reader(f)):
        if i != 0:
            if row[1] not in code_id_journal:
                code_id_journal[row[0]] = row[1]

In [34]:
#calaulate number of papers accorinding to journals
journal_papers = {}
with open('/Users/shemingrong/PROJECT1_gender_differences/aps/publications.csv', 'r') as f:
    for i, row in enumerate(csv.reader(f)):
        if i != 0:
            if code_id_journal[row[1]] not in journal_papers:
                journal_papers[code_id_journal[row[1]]] = 1
            else:
                journal_papers[code_id_journal[row[1]]] +=1

In [35]:
authorships_df = pd.read_csv('/Users/shemingrong/PROJECT1_gender_differences/aps/authorships.csv')

In [36]:
# extract {id_publication: set(id_author)} and {id_author: set(publication)}
authorships_dict = {}
author_papers = {}
author_publish_year = {}
with open('/Users/shemingrong/PROJECT1_gender_differences/aps/authorships.csv', 'r') as f:
    for i, row in enumerate(csv.reader(f)):
        if i != 0:
            if row[1] not in authorships_dict:
                authorships_dict[row[1]] = {author_id_dict[row[2]]}
            else:
                authorships_dict[row[1]].add(author_id_dict[row[2]])
           
            if author_id_dict[row[2]] not in author_papers:
                author_papers[author_id_dict[row[2]]] = {row[1]}
            else:
                author_papers[author_id_dict[row[2]]].add(row[1])
                
            if author_id_dict[row[2]] not in author_publish_year:
                author_publish_year[author_id_dict[row[2]]] = [publication_year_dict[row[1]]]
            else:
                author_publish_year[author_id_dict[row[2]]].append(publication_year_dict[row[1]])
                   

In [37]:
with open('/Users/shemingrong/PROJECT1_gender_differences/total_author_infos.csv', 'w') as fw:
    writer = csv.writer(fw)
    writer.writerow(['id_author_name', "id_author", "name", 'doi', 'coauthors', "year", "gender"])
    with open('/Users/shemingrong/PROJECT1_gender_differences/aps/authorships.csv', 'r') as f:
        for i, row in enumerate(csv.reader(f)):
            if i != 0:
                authors = authorships_dict[row[1]]
                coauthor = set()
                for author in authors:
                    if author != author_id_dict[row[2]]:
                        coauthor.add(author)
                writer.writerow([row[2], author_id_dict[row[2]], author_names_dict[row[2]], publication_id_dict[row[1]], coauthor, publication_year_dict[row[1]], author_gender_dict[author_id_dict[row[2]]]])

# get focal authors

In [9]:
#calculate career length of 869,679 authors
author_career_length = {}
for author in author_publish_year:
    if author not in author_career_length:
        start = datetime.datetime.strptime(min(author_publish_year[author]), "%Y-%m-%d")
        end = datetime.datetime.strptime(max(author_publish_year[author]), "%Y-%m-%d")
        interval = relativedelta(end, start).years
        if interval != 0:
            author_career_length[author] = interval
        else:
            author_career_length[author] = 0.5

#calculate coauthors of 869,679 authors
author_coauthors = {}
for author in author_papers:
    coauthors = set()
    for paper in author_papers[author]:
        for x in authorships_dict[paper]:
            if x != author:
                coauthors.add(x)
    author_coauthors[author] = coauthors

In [12]:
focal_authors = set()
for author in author_papers:
    if author_gender_dict[author] != 'unknown':
        if 3 < len(author_papers[author]) < 400:
            if 3 < author_career_length[author] < 50:
                if len(author_coauthors[author]) > 1:
                    end = datetime.datetime.strptime(max(author_publish_year[author]), "%Y-%m-%d")
                    end_year = int(max(author_publish_year[author]).split('-')[0])
                    if end_year < 2015:
                        focal_authors.add(author)
print(len(focal_authors))

23327


In [14]:
class Paper:
    name_abbr = {"PR": "PhysRev", "PRA": "PhysRevA", "PRAB": "PhysRevAccelBeams", "PRAPPLIED": "PhysRevApplied",
                 "PRB": "PhysRevB", "PRC": "PhysRevC", "PRD": "PhysRevD", "PRE": "PhysRevE",
                 "PRFLUIDS": "PhysRevFluids", "PRI": "PhysRevSeriesI", "PRL": "PhysRevLett",
                 "PRMATERIALS": "PhysRevMaterials", "PRPER": "PhysRevPhysEducRes", "PRRESEARCH": "PhysRevResearch",
                 "PRSTAB": "PhysRevSTAB", "PRSTPER": "PhysRevSTPER", "PRX": "PhysRevX", "PRXQUANTUM": "PRXQuantum",
                 "RMP": "RevModPhys"}

    def __init__(self, doi):
        self.doi = doi
        self.title = None
        self.journal = None
        self.volume = None
        self.date = None
        self.numPages = None
        self.articleType = None
        self.authors = None
        self.affiliations = None
        self.year=None
        self.journal_abbr = {v: k for k, v in self.name_abbr.items()}  # reverse name map
        self.exist = self.getPaper()

    def getPaper(self):
        info = self.doi.split('/')[1].split('.')
        self.journal = info[0]
        self.volume = info[1]
        if self.journal not in self.journal_abbr:
            return False
        path = "/Users/shemingrong/PROJECT1_gender_differences/aps-dataset-metadata-2020/" + self.journal_abbr[self.journal] + '/' + \
               str(self.volume) + "/" + '.'.join(info) + ".json"
        if os.path.exists(path) is False:
            return False
        with open(path, 'r', encoding='utf-8') as f:
            paper_json = json.load(f)
            self.title = paper_json['title']['value']
            self.date = paper_json['date']
            self.year = paper_json['date'].split('-')[0]
            self.numPages = paper_json['numPages'] if "numPages" in paper_json else None
            self.articleType = paper_json['articleType'] if "articleType" in paper_json else None
            if "authors" in paper_json:
                self.numAuthor = len(paper_json['authors'])
                self.authors = paper_json['authors']
                surnames = []
                for author in self.authors:
                    if "surname" in author.keys():
                        surnames.append(author["surname"].lower())
                if len(self.authors) >= 2 and surnames == sorted(surnames):
                    self.is_alpha = True
                else:
                    self.is_alpha = False
            self.affiliations = paper_json['affiliations'] if "affiliations" in paper_json else None
        return True

    def printPaperInfo(self):
        print(self.doi, end=' ')
        print(self.title)
        print(self.date, end=' ')
        print(str(self.numPages) + "pages", self.articleType)
        print(self.authors)
        print(self.affiliations)

    def __str__(self):
        return json.dumps({'doi': self.doi, 'authors': self.authors})

In [15]:
class Author:
    def __init__(self, name, doi):
        self.name = name
        self.doi = doi
        paper = Paper(doi)
        self.coauthor = []
        affiliationIds = None
        if len(paper.authors) > 1:
            if paper.authors[-1]['name'] == name:
                self.last_author = True
            else:
                self.last_author = False
        else:
            self.last_author = False

        for author in paper.authors:
            if author['name'] != name:
                self.coauthor.append(author['name'])
            else:
                self.author_name = author['name']
                # find affiliation
                affiliationIds = author["affiliationIds"]
        self.affiliation = []
        # some paper dont have affiliation
        if paper.affiliations is not None:
            affiliation_map = dict()
            for a in paper.affiliations:
                affiliation_map[a['id']] = a['name']
            for id in affiliationIds:
                if id in affiliation_map.keys():
                    self.affiliation.append(affiliation_map[id])
                else:
                    # some user's affiliationId dont have affiliation_map name
                    pass

In [39]:
def get_focal_authors_info():
    with open('/Users/shemingrong/PROJECT1_gender_differences/total_author_infos.csv', 'r') as f:
        with open('/Users/shemingrong/PROJECT1_gender_differences/focal_author_infos.csv', 'w') as fw:
            writer = csv.writer(fw)
            writer.writerow(["id", "name", "gender", 'doi', 'journal', 'is_alpha', "year", 'numAuthor', 'coauthors', 'last_author'])
            for i, row in tqdm(enumerate(csv.reader(f))):
                if i != 0:
                    if row[1] in focal_authors:
                        if row[1] == '137098':
                            # row[2] = row[2] +' '
                            last_author = Author(row[2] +' ', row[3]).last_author
                        else:
                            last_author = Author(row[2], row[3]).last_author
                        writer.writerow([row[1], row[2], row[6], row[3], Paper(row[3]).journal, Paper(row[3]).is_alpha, row[5], Paper(row[3]).numAuthor, row[4], last_author])


In [40]:
get_focal_authors_info()

2386549it [00:54, 43583.30it/s] 


# get PI

In [22]:
name_group = {}
A = set()
source = ['SS', 'À', 'Á', 'Â', 'Ã', 'Ä', 'Å', 'Æ', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 
          'Î', 'Ï', 'İ', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', 'Ø', 'Ù', 'Ú', 'Û', 'Ü', 'Ý', 
          'Ÿ', 'Ā', 'Ă', 'Ą', 'Ć', 'Ĉ', 'Ċ', 'Č', 'Ď', 'Đ', 'Ē', 'Ĕ', 'Ė', 'Ę', 'Ě', 
          'Ğ', 'Ġ', 'Ĩ', 'Ī', 'Ĭ', 'I', 'Ĺ', 'Ľ', 'Ł', 'Ń', 'Ņ', 'Ň', 'ʼN', 'Ō', 'Ŏ',
          'Ő','Œ', 'Ŕ', 'Ř', 'Ś', 'Ŝ', 'Ş', 'Š', 'Ţ', 'Ť', 'Ũ', 'Ū', 'Ů', 'Ű', 'Ź',
          'Ż','Ž', 'Ǎ', 'Ǐ', 'Ǒ', 'Ǧ', 'Ȩ', 'Β', 'Ḡ', 'Ṁ', 'Ṣ', 'Ồ', 'Ỳ', 'ℜ', '¢']

target = ['SS', 'A', 'A', 'A', 'A', 'A', 'A', 'AE', 'C', 'E', 'E', 'E', 'E', 'I', 'I',
          'I', 'I', 'I', 'N', 'O', 'O', 'O', 'O', 'O', 'O', 'U', 'U', 'U', 'U', 'Y',
          'Y', 'A', 'A', 'A', 'C', 'C', 'C', 'C', 'D', 'D', 'E', 'E', 'E', 'E', 'E',
          'G', 'G', 'I', 'I', 'I', 'I', 'L', 'L', 'L', 'N', 'N', 'N', 'N', 'O', 'O',
          'O', 'OE', 'R', 'R', 'S', 'S', 'S', 'S', 'T', 'T', 'U', 'U', 'U', 'U', 'Z',
          'Z', 'Z', 'A', 'I', 'O', 'G', 'E', 'SS', 'G', 'M', 'S', 'O', 'Y', 'R', 'C']

A_M = {'A', 'B', 'C', 'D', 'E', "F", "G", "H", "I", "J", "K", "L", "M"}
N_Z = {'N', "O", "P", "Q", "R", "S", "T", "U", 'V', 'W', 'X', 'Y', 'Z'}
with open('/Users/shemingrong/PROJECT1_gender_differences/focal_author_infos.csv', 'r') as f:
    for i, row in enumerate(csv.reader(f)):
        if i != 0:
            if row[0] not in name_group:
                
                if row[1].split(' ')[-1][0].upper() == '(':
                    first_string = row[1].split(' ')[-2][0].upper()
                else:
                    first_string = row[1].split(' ')[-1][0].upper()
                    for s, t in zip(source, target):
                        first_string = first_string.replace(s, t)                
                if first_string in A_M:
                        name_group[row[0]] = 'A_M'
                        A.add(row[0])
                elif first_string in N_Z:
                        name_group[row[0]] = 'N_Z'  

In [24]:
links_coauthors = {}
with open('/Users/shemingrong/PROJECT1_gender_differences/focal_author_infos.csv', 'r') as f:
    for i, row in enumerate(csv.reader(f)):
        if i != 0:
            row[8] = ast.literal_eval(row[8])
            if row[0] not in links_coauthors:
                links = {}               
                for coauthor in row[8]:
                    links[coauthor] = 1
                links_coauthors[row[0]] = links
            else:
                for coauthor in row[8]:
                    if coauthor not in links_coauthors[row[0]]:
                        links_coauthors[row[0]][coauthor] = 1
                    else:
                        links_coauthors[row[0]][coauthor] += 1
average_links = {}
for id in links_coauthors:
    links = links_coauthors[id]
    average_links[id] = sum(links.values())/len(links)

In [25]:
#get clustering coefficient
author_triangles = {}
with open('/Users/shemingrong/PROJECT1_gender_differences/focal_author_infos.csv', 'r') as f:
    for i, row in enumerate(csv.reader(f)):
        if i != 0:
            row[8] = ast.literal_eval(row[8])
            if row[0] not in author_triangles:
                if len(row[8]) > 1:
                    triangles = list(combinations(row[8], 2))
                    author_triangles[row[0]] = triangles
                else:
                    author_triangles[row[0]] = []
            else:
                if len(row[8]) > 1:
                    triangles = list(combinations(row[8], 2))
                    for triangle in triangles:
                        if triangle not in author_triangles[row[0]]:
                            m = triangle[0]
                            n = triangle[1]
                            if (n, m) not in author_triangles[row[0]]:
                                author_triangles[row[0]].append(triangle) 
cluster_coeff = {}
#extract the connection between co-authors of focal author, Global clustering coefficient
# for author in author_triangles:
#     triangles = len(author_triangles[author])
#     networks = len(author_coauthors[author])
#     cluster_coeff[author] = 3 * triangles / (3 * triangles + math.comb(networks,2) - triangles)
#extract the connection between co-authors of focal author, local clustering coefficient
for author in author_triangles:
    triangles = len(author_triangles[author])
    networks = len(author_coauthors[author])
    cluster_coeff[author] = triangles / math.comb(networks,2)

In [26]:
#get average coauthors
total_coauthors_counts = {}
with open('/Users/shemingrong/PROJECT1_gender_differences/focal_author_infos.csv', 'r') as f:
    for i, row in enumerate(csv.reader(f)):
        if i != 0:
            row[8] = ast.literal_eval(row[8])
            if row [0] not in total_coauthors_counts:
                total_coauthors_counts[row[0]] = list(row[8])
            else:
                total_coauthors_counts[row[0]] = total_coauthors_counts[row[0]] + list(row[8])
average_authors = {}
for id in total_coauthors_counts:
    average_authors[id] = len(total_coauthors_counts[id]) / len(author_papers[id])

In [70]:
last_author_counts = {}
with open('/Users/shemingrong/PROJECT1_gender_differences/focal_author_infos.csv', 'r') as f:
    for i, row in enumerate(csv.reader(f)):
        if i != 0:
            if row[9] == "True":
                if row[5] != "True" or int(row[7]) < 3:
                    if row[0] not in last_author_counts:
                        last_author_counts[row[0]] = [row[6]]
                    else:
                        last_author_counts[row[0]].append(row[6])
                        
after_principal_investigator = {}
for author in last_author_counts:
    if len(last_author_counts[author]) >= 3:
        after_principal_investigator[author] = sorted(last_author_counts[author])[2]
print(len(after_principal_investigator))#27384

with open('/Users/shemingrong/PROJECT1_gender_differences/drop_focal_authors_data.csv', 'w') as fw:
    writer = csv.writer(fw)
    writer.writerow(['author.name',
     'no.of.papers',
     'Gender',
     'indegree',
     'average.links',
     'clusteringcoefficient.global',
     'average.coauthors',
     'first.year',
     'last.year',
     'third.lastauthor',
     'n.years',
     'Time.to.PI',
     'nyears.PI', 
     "alpha.group"])
    for author in focal_authors:
        npapers = len(author_papers[author])
        gender = author_gender_dict[author]
        indegree = len(author_coauthors[author])
        strength = npapers/indegree
        cc = cluster_coeff[author]
        triangles = len(author_triangles[author])
        demoninator = math.comb(indegree,2)
        first_year = min(author_publish_year[author])
        last_year = max(author_publish_year[author])
        nyears = author_career_length[author]
        if author in after_principal_investigator:
            first_lastauthor = sorted(last_author_counts[author])[0]
            second_lastauthor = sorted(last_author_counts[author])[1]
            third_lastauthor = sorted(last_author_counts[author])[2]
#             last_author_gap = relativedelta(datetime.datetime.strptime(third_lastauthor,"%Y-%m-%d"), datetime.datetime.strptime(first_lastauthor,"%Y-%m-%d")).years
            if third_lastauthor == first_year:
                time_to_pi = 0.5
            else:
                time_to_pi = relativedelta(datetime.datetime.strptime(third_lastauthor,"%Y-%m-%d"), datetime.datetime.strptime(first_year,"%Y-%m-%d")).years
                if time_to_pi == 0:
                    time_to_pi = 0.5
            if third_lastauthor == last_year:
                nyears_PI = 0.5
            else:
                nyears_PI = relativedelta(datetime.datetime.strptime(last_year,"%Y-%m-%d"), datetime.datetime.strptime(third_lastauthor,"%Y-%m-%d")).years
                if nyears_PI == 0:
                    nyears_PI = 0.5
        else:
            third_lastauthor = ''
            last_author_gap = ''
            time_to_pi = ''
            nyears_PI = ''
            if author in last_author_counts:
                first_lastauthor = sorted(last_author_counts[author])[0]
                if len(last_author_counts[author]) == 2:
                    second_lastauthor = sorted(last_author_counts[author])[1]
                else:
                    second_lastauthor = ''
            else:
                first_lastauthor = ''
                second_lastauthor = ''
                third_lastauthor = ''
        alpha_group = name_group[author]
        writer.writerow([author, 
                         npapers, 
                         gender, 
                         indegree, 
                         average_links[author], 
                         cc, 
                         average_authors[author], 
                         first_year, 
                         last_year, 
                         third_lastauthor,
                         nyears,
                         time_to_pi, 
                         nyears_PI, 
                         alpha_group])     

8797


In [69]:
#get PI author before remove alphabetical
last_author_counts = {}
with open('/Users/shemingrong/PROJECT1_gender_differences/focal_author_infos.csv', 'r') as f:
    for i, row in enumerate(csv.reader(f)):
        if i != 0:
            if row[9] == "True":
                if row[0] not in last_author_counts:
                    last_author_counts[row[0]] = [row[6]]
                else:
                    last_author_counts[row[0]].append(row[6])
before_principal_investigator = {}
for author in last_author_counts:
    if len(last_author_counts[author]) >= 3:
        before_principal_investigator[author] = sorted(last_author_counts[author])[2]
print(len(before_principal_investigator))#27384

with open('/Users/shemingrong/PROJECT1_gender_differences/drop_focal_authors_data_without_alpha.csv', 'w') as fw:
    writer = csv.writer(fw)
    writer.writerow(['author.name',
     'no.of.papers',
     'Gender',
     'indegree',
     'average.links',
     'clusteringcoefficient.global',
     'average.coauthors',
     'first.year',
     'last.year',
     'third.lastauthor',
     'n.years',
     'Time.to.PI',
     'nyears.PI', 
     "alpha.group"])
    for author in focal_authors:
        npapers = len(author_papers[author])
        gender = author_gender_dict[author]
        indegree = len(author_coauthors[author])
        strength = npapers/indegree
        cc = cluster_coeff[author]
        triangles = len(author_triangles[author])
        demoninator = math.comb(indegree,2)
        first_year = min(author_publish_year[author])
        last_year = max(author_publish_year[author])
        nyears = author_career_length[author]
        if author in before_principal_investigator:
            first_lastauthor = sorted(last_author_counts[author])[0]
            second_lastauthor = sorted(last_author_counts[author])[1]
            third_lastauthor = sorted(last_author_counts[author])[2]
#             last_author_gap = relativedelta(datetime.datetime.strptime(third_lastauthor,"%Y-%m-%d"), datetime.datetime.strptime(first_lastauthor,"%Y-%m-%d")).years
            if third_lastauthor == first_year:
                time_to_pi = 0.5
            else:
                time_to_pi = relativedelta(datetime.datetime.strptime(third_lastauthor,"%Y-%m-%d"), datetime.datetime.strptime(first_year,"%Y-%m-%d")).years
                if time_to_pi == 0:
                    time_to_pi = 0.5
            if third_lastauthor == last_year:
                nyears_PI = 0.5
            else:
                nyears_PI = relativedelta(datetime.datetime.strptime(last_year,"%Y-%m-%d"), datetime.datetime.strptime(third_lastauthor,"%Y-%m-%d")).years
                if nyears_PI == 0:
                    nyears_PI = 0.5
        else:
            third_lastauthor = ''
            last_author_gap = ''
            time_to_pi = ''
            nyears_PI = ''
            if author in last_author_counts:
                first_lastauthor = sorted(last_author_counts[author])[0]
                if len(last_author_counts[author]) == 2:
                    second_lastauthor = sorted(last_author_counts[author])[1]
                else:
                    second_lastauthor = ''
            else:
                first_lastauthor = ''
                second_lastauthor = ''
                third_lastauthor = ''
        alpha_group = name_group[author]
        
        writer.writerow([author, 
                         npapers, 
                         gender, 
                         indegree, 
                         average_links[author], 
                         cc, 
                         average_authors[author], 
                         first_year, 
                         last_year, 
                         third_lastauthor,
                         nyears,
                         time_to_pi, 
                         nyears_PI, 
                         alpha_group])     

9811
